In [64]:
import pandas as pd
import yaml
from sqlalchemy import create_engine

## Database Connection

In [65]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [66]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [67]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction

In [68]:
cliente = pd.read_sql_table('dim_cliente', etl_conn)
mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
fecha = pd.read_sql_table('dim_fecha', etl_conn)
servicio = pd.read_sql_table("mensajeria_servicio",url_db)

In [69]:
total_nan = servicio.isna().sum()
print(total_nan)

id                                    0
descripcion                           0
nombre_solicitante                    0
fecha_solicitud                       0
hora_solicitud                        0
fecha_deseada                         6
hora_deseada                          0
nombre_recibe                         0
telefono_recibe                       0
descripcion_pago                    260
ida_y_regreso                         0
activo                                0
novedades                           336
cliente_id                            0
destino_id                            0
mensajero_id                        727
origen_id                             0
tipo_pago_id                          0
tipo_servicio_id                      0
tipo_vehiculo_id                      0
usuario_id                            0
prioridad                             0
ciudad_destino_id                     0
ciudad_origen_id                      0
hora_visto_por_mensajero          28428


In [70]:
print(len(servicio))

28430


In [71]:
servicio.head(5)

,id,descripcion,nombre_solicitante,fecha_solicitud,hora_solicitud,fecha_deseada,hora_deseada,nombre_recibe,telefono_recibe,descripcion_pago,...,ciudad_origen_id,hora_visto_por_mensajero,visto_por_mensajero,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado
0,34,recoger muestras a las 10 am,chat_GPT,2023-10-26,09:46:03,2023-10-26,09:46:03,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,
1,35,Recogervmx a las,chat_GPT,2023-10-26,11:18:14,2023-10-26,11:18:14,Gemini,100,,...,1,None,,,NaN,NaN,False,False,True,
2,36,recoger mx a las 9 am,chat_GPT,2023-10-28,19:21:01,2023-10-30,19:21:01,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,
3,41,recoger mx a las 11 am,chat_GPT,2023-11-07,09:46:09,2023-11-07,09:46:09,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,
4,42,recoger mx a las 11 am,chat_GPT,2023-11-07,09:46:10,2023-11-07,09:46:10,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,


## Transformation

In [72]:
fecha_unique = fecha[["month_year", "key_fecha"]].drop_duplicates(subset=["month_year"])
servicio["año_mes"] = servicio["fecha_solicitud"].dt.to_period("M").astype(str)
servicio = servicio[["id", "año_mes", "cliente_id", "mensajero_id"]]
servicio.fillna(-1, inplace=True)
# Hacer el merge asegurando que coincidan
hecho_servicio_mensual = pd.merge(servicio, fecha_unique[["month_year", "key_fecha"]], left_on="año_mes", right_on="month_year", how="left")
hecho_servicio_mensual.head(5)

,id,año_mes,cliente_id,mensajero_id,month_year,key_fecha
0,34,2023-10,5,-1.0,2023-10,265
1,35,2023-10,5,7.0,2023-10,265
2,36,2023-10,5,-1.0,2023-10,265
3,41,2023-11,5,-1.0,2023-11,296
4,42,2023-11,5,-1.0,2023-11,296


In [73]:
hecho_servicio_mensual = pd.merge(hecho_servicio_mensual, cliente[["cliente_id", "key_cliente"]], left_on="cliente_id", right_on="cliente_id", how="left")
hecho_servicio_mensual.head(5)

,id,año_mes,cliente_id,mensajero_id,month_year,key_fecha,key_cliente
0,34,2023-10,5,-1.0,2023-10,265,7
1,35,2023-10,5,7.0,2023-10,265,7
2,36,2023-10,5,-1.0,2023-10,265,7
3,41,2023-11,5,-1.0,2023-11,296,7
4,42,2023-11,5,-1.0,2023-11,296,7


In [74]:
mensajero.rename(columns={'id':'mensajero_id'}, inplace=True)
hecho_servicio_mensual = hecho_servicio_mensual.merge(mensajero[["mensajero_id", "key_mensajero"]], left_on="mensajero_id", right_on="mensajero_id", how="left")
hecho_servicio_mensual.fillna(-1, inplace=True)
hecho_servicio_mensual.head(5)

,id,año_mes,cliente_id,mensajero_id,month_year,key_fecha,key_cliente,key_mensajero
0,34,2023-10,5,-1.0,2023-10,265,7,-1.0
1,35,2023-10,5,7.0,2023-10,265,7,13.0
2,36,2023-10,5,-1.0,2023-10,265,7,-1.0
3,41,2023-11,5,-1.0,2023-11,296,7,-1.0
4,42,2023-11,5,-1.0,2023-11,296,7,-1.0


In [75]:
print(len(hecho_servicio_mensual))

28430


## Cálculo del atributo solicitudes_por_mes

In [76]:
# Contar solicitudes por cada 'año_mes'
solicitudes_por_mes = hecho_servicio_mensual.groupby("año_mes").size().reset_index(name="solicitud_por_mes")
solicitudes_por_mes
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original
hecho_servicio_mensual = hecho_servicio_mensual.merge(solicitudes_por_mes, on="año_mes", how="left")
hecho_servicio_mensual.head(5)

,id,año_mes,cliente_id,mensajero_id,month_year,key_fecha,key_cliente,key_mensajero,solicitud_por_mes
0,34,2023-10,5,-1.0,2023-10,265,7,-1.0,12
1,35,2023-10,5,7.0,2023-10,265,7,13.0,12
2,36,2023-10,5,-1.0,2023-10,265,7,-1.0,12
3,41,2023-11,5,-1.0,2023-11,296,7,-1.0,17
4,42,2023-11,5,-1.0,2023-11,296,7,-1.0,17


In [77]:
print(len(hecho_servicio_mensual))

28430


## Cálculo del atributo solicitudes_por_mes_cliente

In [78]:
# Contar solicitudes por cada 'key_fecha' y 'key_cliente' 
solicitud_por_mes_cliente = hecho_servicio_mensual.groupby(["key_fecha","key_cliente"]).size().reset_index(name="solicitud_por_mes_cliente")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original
hecho_servicio_mensual = hecho_servicio_mensual.merge(solicitud_por_mes_cliente, on=["key_cliente", "key_fecha"], how="left")
hecho_servicio_mensual.head(5)

,id,año_mes,cliente_id,mensajero_id,month_year,key_fecha,key_cliente,key_mensajero,solicitud_por_mes,solicitud_por_mes_cliente
0,34,2023-10,5,-1.0,2023-10,265,7,-1.0,12,12
1,35,2023-10,5,7.0,2023-10,265,7,13.0,12,12
2,36,2023-10,5,-1.0,2023-10,265,7,-1.0,12,12
3,41,2023-11,5,-1.0,2023-11,296,7,-1.0,17,16
4,42,2023-11,5,-1.0,2023-11,296,7,-1.0,17,16


In [79]:
print(len(hecho_servicio_mensual))

28430


## Cálculo del atributo solicitudes_por_mensajero

In [80]:
# Contar solicitudes por cada 'key_fecha' y 'key_mensajero'
cantidad_solicitud_por_mensajero = hecho_servicio_mensual.groupby(["key_fecha","key_mensajero"]).size().reset_index(name="cantidad_solicitud_por_mes_mensajero")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original
hecho_servicio_mensual = hecho_servicio_mensual.merge(cantidad_solicitud_por_mensajero, on=["key_fecha","key_mensajero"], how="left")
hecho_servicio_mensual.head(6)

,id,año_mes,cliente_id,mensajero_id,month_year,key_fecha,key_cliente,key_mensajero,solicitud_por_mes,solicitud_por_mes_cliente,cantidad_solicitud_por_mes_mensajero
0,34,2023-10,5,-1.0,2023-10,265,7,-1.0,12,12,5
1,35,2023-10,5,7.0,2023-10,265,7,13.0,12,12,7
2,36,2023-10,5,-1.0,2023-10,265,7,-1.0,12,12,5
3,41,2023-11,5,-1.0,2023-11,296,7,-1.0,17,16,8
4,42,2023-11,5,-1.0,2023-11,296,7,-1.0,17,16,8
5,43,2023-11,5,-1.0,2023-11,296,7,-1.0,17,16,8


## Eliminar filas no necesarias

In [81]:
hecho_servicio_mensual.drop(columns=["id","cliente_id","año_mes","mensajero_id","month_year"], inplace=True)
hecho_servicio_mensual

,key_fecha,key_cliente,key_mensajero,solicitud_por_mes,solicitud_por_mes_cliente,cantidad_solicitud_por_mes_mensajero
0,265,7,-1.0,12,12,5
1,265,7,13.0,12,12,7
2,265,7,-1.0,12,12,5
3,296,7,-1.0,17,16,8
4,296,7,-1.0,17,16,8
...,...,...,...,...,...,...
28425,570,11,8.0,4304,2782,138
28426,570,11,27.0,4304,2782,206
28427,570,7,32.0,4304,628,335
28428,570,11,18.0,4304,2782,73


## Eliminar Datos Duplicados

In [82]:
hecho_servicio_mensual_sin_duplicados = hecho_servicio_mensual.drop_duplicates()

In [83]:
len(hecho_servicio_mensual_sin_duplicados)

890

## Loading

In [84]:
hecho_servicio_mensual_sin_duplicados.to_sql("hecho_servicio_mensual", etl_conn, if_exists="replace", index_label="key_solicitud_servicio") 

890